In [10]:
#SCRAPING ENVOIREMENT

In [2]:
#Importing libaries
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os
import re
import itertools
import random
import threading
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
import csv

## scraping of links of first 200 pages of links

In [447]:
#Importing page-links

list_url = []
for page in range(1,201,1):
    url = f'https://www.boligsiden.dk/tilsalg/villa,ejerlejlighed,raekkehus,landejendom?page={page}'
    list_url.append(url)

print(len(list_url))





200


In [449]:
#Retriving HTMLS

list_htmls =[]
for url in list_url:
    response = requests.get(url, headers={'name': 'Navn', 'email': 'ku_id@alumni.ku.dk'})
    list_htmls.append(response.text)
    time.sleep(0.5)

    

In [451]:
# Finding links on appartments

links = []
for html in list_htmls:
    soup = BeautifulSoup(html, 'lxml')
    pattern = re.compile(r'href\":\"(/addresses/[^\"]+)')
    matches = pattern.findall(str(soup))

    apartments_list = list(set(matches))


    base_url = "https://www.boligsiden.dk"
    apartment_links = [base_url + link for link in apartments_list] #This list contains appartments
    links.append(apartment_links)

flattened_links = list(itertools.chain(*links))
print(len(flattened_links)) #Counting number of links

9996


In [453]:
#Saving list of links to be scrapped

with open('rawdata/links.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    for link in flattened_links:
        writer.writerow([link])


# HER STARTER SCRAPING PROCESSEN

In [4]:
#Importing relevant list of links:
flattened_links = []

with open('rawdata/links_version_2.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        flattened_links.append(row[0])  # Assuming each link is in the first column

print(len(flattened_links))

9999


In [6]:
op = webdriver.ChromeOptions()
op.add_argument('headless')
driver = webdriver.Chrome(options=op)

In [24]:
#Carefull! Runs with X-links at a time
#NANNAS VERSION
dict = {}
links_med_fejl = []

def fetch_data(link):
    driver = webdriver.Chrome(options=op)
    try:
        driver.get(link)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        time.sleep(random.uniform(2, 3))  # Reducing sleep time slightly
        area_size = soup.find_all('span', class_='text-gray-800 whitespace-nowrap cursor-pointer border-b border-gray-400 pb-0.5 border-dotted text-sm')
        other = soup.find_all('span', class_="text-gray-800 whitespace-nowrap border-dotted text-sm")
        address = soup.find_all('div', class_='font-bold text-sm md:text-base')
        pstnr = soup.find_all('div', class_="mt-1 text-xs md:text-sm text-gray-600")
        pstnr_ny = re.findall(r'\d+', pstnr[0].get_text())  # Searching for numbers in postal-code string.
        try:
            total_address = address[0].get_text() + ',' + pstnr_ny[0]
        except:
            print('adresseproblem')
        #Nannas kode:
        try:    
            price = soup.find('h2', class_='text-2xl xl:text-3xl text-blue-900 font-bold').get_text()
        except:
            price = "fejl"
            
        #Finder label hvor afstand er under

        try:
            boligtype = soup.find('span', class_='text-black text-sm pr-2').get_text()
        except:
            boligtype = "fejl"
        #energi = soup.find('svg', id='Lag_1').find('title')
        try:
            energi = soup.find('title', string=re.compile('Energimærke')).get_text()
        except:
            energi = "fejl"
        try:
            radon = soup.find('div', class_='h-7 text-sm font-bold').get_text()
        except:
            radon = "fejl"
    
        #kommune_info og liggetid med text behandling
        try:
            kommune_info = soup.find('div', class_='sec_municipality_info')
            if kommune_info:
                kommune_text = kommune_info.get_text()
        
                if 'Kommuneskatten er på ' in kommune_text:
                    kommuneskat = kommune_text.split('Kommuneskatten er på ')[1].split(' ')[0]
                else:
                    kommuneskat = "fejl"
        
                if 'grundskyldspromillen på ' in kommune_text:
                    grundskyldspromillen = kommune_text.split('grundskyldspromillen på ')[1].split(' ')[0]
                else:
                    grundskyldspromillen = "fejl"
        except:
            kommuneskat = "fejl"
            grundskyldspromillen = "fejl"
            
            
        
        try:
            liggetid = soup.find('div', class_='pb-0.5 border-dashed border-b border-gray-300 cursor-pointer text-sm text-gray-800')
            if liggetid:
                liggetid_text = liggetid.get_text(separator=" ", strip=True).replace(' :', ':')
        except:
            liggetid_text = "fejl"

        #################   NY KODE med skoler    ########################
        #Skolenavn
        #Skolenavn
        try:
            school_name_1 = soup.find('div', class_='text-sm font-bold text-blue-900').get_text()
        except:
            school_name_1 = "fejl"
        try:
            school_name_2 = soup.find('div', class_='text-sm font-bold text-blue-900').find_next('div', class_='text-sm font-bold text-blue-900').get_text()
        except:
            school_name_2 = "fejl"
        try:
            school_name_3 = soup.find('div', class_='text-sm font-bold text-blue-900').find_next('div', class_='text-sm font-bold text-blue-900').find_next('div', class_='text-sm font-bold text-blue-900').get_text()
        except:
            school_name_3 = "fejl"

        #Afstand 
        try:
            school_1 = soup.find('div', string='Afstand').find_next_sibling('div').get_text()
        except:
            school_1 = "fejl"
        try:
            school_2 = soup.find('div', string='Afstand').find_next('div', string='Afstand').find_next_sibling('div').get_text()
        except:
            school_2 = "fejl"
        try:
            school_3 = soup.find('div', string='Afstand').find_next('div', string='Afstand').find_next('div', string='Afstand').find_next_sibling('div').get_text()
        except:
            school_3 = "fejl"

        #Karakterer
        try:
            grades_1 = soup.find('div', string='Karaktergennemsnit').find_next_sibling('div').find_next_sibling('div').get_text()
        except:
            grades_1 = "fejl"
        try:
            grades_2 = soup.find('div', string='Karaktergennemsnit').find_next('div', string='Karaktergennemsnit').find_next_sibling('div').find_next_sibling('div').get_text()
        except:
            grades_2 = "fejl"
        try:
            grades_3 = soup.find('div', string='Karaktergennemsnit').find_next('div', string='Karaktergennemsnit').find_next('div', string='Karaktergennemsnit').find_next_sibling('div').find_next_sibling('div').get_text()
        except:
            grades_3 = "fejl"

        #Trivsel
        try:
            trivsel_1 = soup.find('div', string='Trivsel').find_next_sibling('div').find_next_sibling('div').get_text()
        except: 
            trivsel_1 = "fejl"
        try:
            trivsel_2 = soup.find('div', string='Trivsel').find_next('div', string='Trivsel').find_next_sibling('div').find_next_sibling('div').get_text()
        except:
            trivsel_2 = "fejl"
        try:
            trivsel_3 = soup.find('div', string='Trivsel').find_next('div', string='Trivsel').find_next('div', string='Trivsel').find_next_sibling('div').find_next_sibling('div').get_text()
        except:
            trivsel_3 = "fejl"

        #ENDELIGE 
        
        if grades_1 not in ["fejl", "Ikke oplyst"]:
            school_name=school_name_1
            school = school_1
            grades = grades_1
            trivsel = trivsel_1
        elif grades_2 not in ["fejl", "Ikke oplyst"]:
            school_name=school_name_2
            school = school_2
            grades = grades_2
            trivsel = trivsel_2
        elif grades_3 not in ["fejl", "Ikke oplyst"]:
            school_name=school_name_3
            school = school_3
            grades = grades_3
            trivsel = trivsel_3
        else:
            school_name=school_name_1
            school = school_1 # Fallback to school_1 if all are "fejl" or "Ikke oplyst"
            grades = grades_1
            trivsel = trivsel_1



        
        try:
            if len(area_size) > 2:  # Bolig
                dict[total_address] = {'Price': price, 'Schoolname': school_name, 'Distance to school': school, 'School-grades': grades, 'Trivsel': trivsel, 'Boligtype': boligtype, 'Energy-classification': energi, 'Radon': radon, 'Kommuneskat': kommuneskat, 'Grundskyldspromillen': grundskyldspromillen, 'Kommune tekst': kommune_text, 'Liggetid': liggetid_text, 'm2': area_size[0].get_text(), 'Grund': area_size[1].get_text(), 'Ejerudgift': area_size[2].get_text(),
                                       'Rooms': other[0].get_text(), 'Build': other[1].get_text(), 'Toilets': other[2].get_text(), 'Postal code': pstnr_ny[0], 'link': link}
            else:  # Lejlighed
                dict[total_address] = {'Price': price, 'Schoolname': school_name, 'Distance to school': school, 'School-grades': grades, 'Trivsel': trivsel, 'Boligtype': boligtype, 'Energy-classification': energi, 'Radon': radon, 'Kommuneskat': kommuneskat, 'Grundskyldspromillen': grundskyldspromillen, 'Kommune tekst': kommune_text, 'Liggetid': liggetid_text, 'm2': area_size[0].get_text(), 'Ejerudgift': area_size[1].get_text(), 'Elevator': other[0].get_text(),
                                       'Rooms': other[1].get_text(), 'Build': other[2].get_text(), 'Toilets': other[3].get_text(),
                                       'Altan': other[4].get_text(), 'Terasse': other[5].get_text(), 'Postal code': pstnr_ny[0], 'link': link}
        except:
            dict[total_address] = {'m2': 'fejl', 'Ejerudgift': 'fejl', 'Elevator': 'fejl', 'Rooms': 'fejl', 'Build': 'fejl', 'Toilet': 'fejl', 'Altan': 'fejl', 'Terasse': 'fejl', 'Postal code': 'fejl', 'link': link}
    except Exception as e:
        print(f'Error occurred for link: {link} - {e}')
        links_med_fejl.append(link)

# Using ThreadPoolExecutor to handle multiple requests concurrently
with ThreadPoolExecutor(max_workers=5) as executor:
    list(tqdm(executor.map(fetch_data, flattened_links[9000:10000]), total=len(flattened_links[9000:10000])))


  1%|▍                                         | 11/999 [00:17<20:34,  1.25s/it]

Error occurred for link: https://www.boligsiden.dk/addresses/0a3f50c6-59e4-32b8-e044-0003ba298018 - list index out of range


  3%|█                                         | 25/999 [00:35<20:58,  1.29s/it]

Error occurred for link: https://www.boligsiden.dk/addresses/0a3f509f-1032-32b8-e044-0003ba298018 - list index out of range
Error occurred for link: https://www.boligsiden.dk/addresses/0a3f50ad-a900-32b8-e044-0003ba298018 - list index out of range
Error occurred for link: https://www.boligsiden.dk/addresses/0a3f50a5-491f-32b8-e044-0003ba298018 - list index out of range


  8%|███▏                                      | 77/999 [02:03<15:44,  1.02s/it]

...

 91%|█████████████████████████████████████▏   | 906/999 [20:34<01:41,  1.09s/it]

Error occurred for link: https://www.boligsiden.dk/addresses/0a3f50c2-206a-32b8-e044-0003ba298018 - list index out of range


 93%|██████████████████████████████████████▏  | 931/999 [21:04<01:29,  1.31s/it]

Error occurred for link: https://www.boligsiden.dk/addresses/0a3f50c6-fd6f-32b8-e044-0003ba298018 - list index out of range
Error occurred for link: https://www.boligsiden.dk/addresses/0a3f50aa-47a5-32b8-e044-0003ba298018 - list index out of range
Error occurred for link: https://www.boligsiden.dk/addresses/0a3f50c0-71a1-32b8-e044-0003ba298018 - list index out of range


100%|█████████████████████████████████████████| 999/999 [22:41<00:00,  1.36s/it]


In [26]:
#Denne celle henter data på alle links med fejl fra ovenstående celle! Kør derfor efter ovenstående.
def fetch_data(link):
    driver = webdriver.Chrome(options=op)
    try:
        driver.get(link)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        time.sleep(random.uniform(2, 3))  # Reducing sleep time slightly
        area_size = soup.find_all('span', class_='text-gray-800 whitespace-nowrap cursor-pointer border-b border-gray-400 pb-0.5 border-dotted text-sm')
        other = soup.find_all('span', class_="text-gray-800 whitespace-nowrap border-dotted text-sm")
        address = soup.find_all('div', class_='font-bold text-sm md:text-base')
        pstnr = soup.find_all('div', class_="mt-1 text-xs md:text-sm text-gray-600")
        pstnr_ny = re.findall(r'\d+', pstnr[0].get_text())  # Searching for numbers in postal-code string.
        try:
            total_address = address[0].get_text() + ',' + pstnr_ny[0]
        except:
            print('adresseproblem')
        #Nannas kode:
        try:    
            price = soup.find('h2', class_='text-2xl xl:text-3xl text-blue-900 font-bold').get_text()
        except:
            price = "fejl"
    
        try:
            boligtype = soup.find('span', class_='text-black text-sm pr-2').get_text()
        except:
            boligtype = "fejl"
        #energi = soup.find('svg', id='Lag_1').find('title')
        try:
            energi = soup.find('title', string=re.compile('Energimærke')).get_text()
        except:
            energi = "fejl"
        try:
            radon = soup.find('div', class_='h-7 text-sm font-bold').get_text()
        except:
            radon = "fejl"
    
        #kommune_info og liggetid med text behandling
        try:
            kommune_info = soup.find('div', class_='sec_municipality_info')
            if kommune_info:
                kommune_text = kommune_info.get_text()
        
                if 'Kommuneskatten er på ' in kommune_text:
                    kommuneskat = kommune_text.split('Kommuneskatten er på ')[1].split(' ')[0]
                else:
                    kommuneskat = "fejl"
        
                if 'grundskyldspromillen på ' in kommune_text:
                    grundskyldspromillen = kommune_text.split('grundskyldspromillen på ')[1].split(' ')[0]
                else:
                    grundskyldspromillen = "fejl"
        except:
            kommuneskat = "fejl"
            grundskyldspromillen = "fejl"
            
            
        
        try:
            liggetid = soup.find('div', class_='pb-0.5 border-dashed border-b border-gray-300 cursor-pointer text-sm text-gray-800')
            if liggetid:
                liggetid_text = liggetid.get_text(separator=" ", strip=True).replace(' :', ':')
        except:
            liggetid_text = "fejl"

        
        #################   NY KODE med skoler    ########################
        #Skolenavn
        try:
            school_name_1 = soup.find('div', class_='text-sm font-bold text-blue-900').get_text()
        except:
            school_name_1 = "fejl"
        try:
            school_name_2 = soup.find('div', class_='text-sm font-bold text-blue-900').find_next('div', class_='text-sm font-bold text-blue-900').get_text()
        except:
            school_name_2 = "fejl"
        try:
            school_name_3 = soup.find('div', class_='text-sm font-bold text-blue-900').find_next('div', class_='text-sm font-bold text-blue-900').find_next('div', class_='text-sm font-bold text-blue-900').get_text()
        except:
            school_name_3 = "fejl"

        #Afstand 
        try:
            school_1 = soup.find('div', string='Afstand').find_next_sibling('div').get_text()
        except:
            school_1 = "fejl"
        try:
            school_2 = soup.find('div', string='Afstand').find_next('div', string='Afstand').find_next_sibling('div').get_text()
        except:
            school_2 = "fejl"
        try:
            school_3 = soup.find('div', string='Afstand').find_next('div', string='Afstand').find_next('div', string='Afstand').find_next_sibling('div').get_text()
        except:
            school_3 = "fejl"

        #Karakterer
        try:
            grades_1 = soup.find('div', string='Karaktergennemsnit').find_next_sibling('div').find_next_sibling('div').get_text()
        except:
            grades_1 = "fejl"
        try:
            grades_2 = soup.find('div', string='Karaktergennemsnit').find_next('div', string='Karaktergennemsnit').find_next_sibling('div').find_next_sibling('div').get_text()
        except:
            grades_2 = "fejl"
        try:
            grades_3 = soup.find('div', string='Karaktergennemsnit').find_next('div', string='Karaktergennemsnit').find_next('div', string='Karaktergennemsnit').find_next_sibling('div').find_next_sibling('div').get_text()
        except:
            grades_3 = "fejl"

        #Trivsel
        try:
            trivsel_1 = soup.find('div', string='Trivsel').find_next_sibling('div').find_next_sibling('div').get_text()
        except: 
            trivsel_1 = "fejl"
        try:
            trivsel_2 = soup.find('div', string='Trivsel').find_next('div', string='Trivsel').find_next_sibling('div').find_next_sibling('div').get_text()
        except:
            trivsel_2 = "fejl"
        try:
            trivsel_3 = soup.find('div', string='Trivsel').find_next('div', string='Trivsel').find_next('div', string='Trivsel').find_next_sibling('div').find_next_sibling('div').get_text()
        except:
            trivsel_3 = "fejl"

        #ENDELIGE 
        
        if grades_1 not in ["fejl", "Ikke oplyst"]:
            school_name=school_name_1
            school = school_1
            grades = grades_1
            trivsel = trivsel_1
        elif grades_2 not in ["fejl", "Ikke oplyst"]:
            school_name=school_name_2
            school = school_2
            grades = grades_2
            trivsel = trivsel_2
        elif grades_3 not in ["fejl", "Ikke oplyst"]:
            school_name=school_name_3
            school = school_3
            grades = grades_3
            trivsel = trivsel_3
        else:
            school_name=school_name_1
            school = school_1 # Fallback to school_1 if all are "fejl" or "Ikke oplyst"
            grades = grades_1
            trivsel = trivsel_1


        
        try:
            if len(area_size) > 2:  # Bolig
                dict[total_address] = {'Price': price, 'Schoolname': school_name, 'Distance to school': school, 'School-grades': grades, 'Trivsel': trivsel, 'Boligtype': boligtype, 'Energy-classification': energi, 'Radon': radon, 'Kommuneskat': kommuneskat, 'Grundskyldspromillen': grundskyldspromillen, 'Kommune tekst': kommune_text, 'Liggetid': liggetid_text, 'm2': area_size[0].get_text(), 'Grund': area_size[1].get_text(), 'Ejerudgift': area_size[2].get_text(),
                                       'Rooms': other[0].get_text(), 'Build': other[1].get_text(), 'Toilets': other[2].get_text(), 'Postal code': pstnr_ny[0], 'link': link}
            else:  # Lejlighed
                dict[total_address] = {'Price': price, 'Schoolname': school_name, 'Distance to school': school, 'School-grades': grades, 'Trivsel': trivsel, 'Boligtype': boligtype, 'Energy-classification': energi, 'Radon': radon, 'Kommuneskat': kommuneskat, 'Grundskyldspromillen': grundskyldspromillen, 'Kommune tekst': kommune_text, 'Liggetid': liggetid_text, 'm2': area_size[0].get_text(), 'Ejerudgift': area_size[1].get_text(), 'Elevator': other[0].get_text(),
                                       'Rooms': other[1].get_text(), 'Build': other[2].get_text(), 'Toilets': other[3].get_text(),
                                       'Altan': other[4].get_text(), 'Terasse': other[5].get_text(), 'Postal code': pstnr_ny[0], 'link': link}
        except:
            dict[total_address] = {'m2': 'fejl', 'Ejerudgift': 'fejl', 'Elevator': 'fejl', 'Rooms': 'fejl', 'Build': 'fejl', 'Toilet': 'fejl', 'Altan': 'fejl', 'Terasse': 'fejl', 'Postal code': 'fejl', 'link': link}
    except Exception as e:
        print(f'Error occurred for link: {link} - {e}')
        links_med_fejl.append(link)

# Using ThreadPoolExecutor to handle multiple requests concurrently
with ThreadPoolExecutor(max_workers=1) as executor:
    list(tqdm(executor.map(fetch_data, links_med_fejl), total=len(links_med_fejl)))


 34%|██████████████▊                            | 11/32 [00:59<01:49,  5.23s/it]

Error occurred for link: https://www.boligsiden.dk/addresses/0a3f50bc-2f96-32b8-e044-0003ba298018 - list index out of range


 56%|████████████████████████▏                  | 18/32 [01:39<01:16,  5.49s/it]

Error occurred for link: https://www.boligsiden.dk/addresses/0a3f50c4-5b19-32b8-e044-0003ba298018 - list index out of range


100%|███████████████████████████████████████████| 32/32 [03:16<00:00,  6.13s/it]


## Lav først til DataFrame og efterfølgende til CSV fil

In [28]:
#Converting dict to dataframe:
DF_NAME = 'index_19000_20000' #Her kalder du din Dataframe/CSV-fil noget unikt.

df_DF_NAME = pd.DataFrame.from_dict(dict, orient='index')
pd.set_option('display.max_colwidth', None)
df_DF_NAME



,Price,Schoolname,Distance to school,School-grades,Trivsel,Boligtype,Energy-classification,Radon,Kommuneskat,Grundskyldspromillen,...,Ejerudgift,Rooms,Build,Toilets,Postal code,link,Elevator,Altan,Terasse,Toilet
"Enebærvænget 14,4700",1.595.000 kr.,Sydbyskolen,"1,83 km",Ikke oplyst,Ikke oplyst,Villa,Energimærke C,Høj,25%,25‰.,...,Ejerudg.: 1.850 kr/md,4 værelser,Opført 1971,1 toilet,4700,https://www.boligsiden.dk/addresses/0a3f50b0-71bc-32b8-e044-0003ba298018,NaN,NaN,NaN,NaN
"Nordre Strandvej 27,8700",1.698.000 kr.,Horsens Byskole,"1,29 km",Ikke oplyst,Ikke oplyst,Villa,Energimærke C,Høj,"25,39%","22,56‰.",...,Ejerudg.: 1.848 kr/md,5 værelser,Opført 1877,1 toilet,8700,https://www.boligsiden.dk/addresses/0a3f50bb-7735-32b8-e044-0003ba298018,NaN,NaN,NaN,NaN
"Øresunds Alle 33,2791",25.500.000 kr.,Dragør Skole,521 m,8.1,3.8,Villa,Energimærke A2015,Mellem,"24,8%","24,47‰.",...,Ejerudg.: 8.462 kr/md,6 værelser,Opført 2018,5 toiletter,2791,https://www.boligsiden.dk/addresses/0a3f50a3-98a8-32b8-e044-0003ba298018,NaN,NaN,NaN,NaN
"Kongelysvej 19,2640",2.995.000 kr.,Læringshuset,280 m,6.5,3.4,Rækkehus,Energimærke A2015,Høj,"24,6%","24,4‰.",...,Ejerudg.: 2.747 kr/md,3 værelser,Opført 2018,1 toilet,2640,https://www.boligsiden.dk/addresses/1fda6ad3-6ab3-41d4-9f36-8eafe8fe3463,NaN,NaN,NaN,NaN
"Peter Heises Kvarter 9,9700",1.795.000 kr.,Søndergades Skole,"1,42 km",Ikke oplyst,Ikke oplyst,Villa,Energimærke C,Mellem,"26,3%","31,23‰.",...,Ejerudg.: 1.851 kr/md,4 værelser,Opført 1999,2 toiletter,9700,https://www.boligsiden.dk/addresses/0a3f50c6-f55d-32b8-e044-0003ba298018,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Rishøjvej 17,8983",895.000 kr.,Grønhøjskolen,"4,94 km",6.2,3.6,Villa,Energimærke F,Høj,"25,8%","27,93‰.",...,Ejerudg.: 1.459 kr/md,5 værelser,Opført 1911,1 toilet,8983,https://www.boligsiden.dk/addresses/0a3f50c0-71a1-32b8-e044-0003ba298018,NaN,NaN,NaN,NaN
"Vesterhavsgade 67, st. 3.,6700",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,fejl,fejl,fejl,NaN,fejl,https://www.boligsiden.dk/addresses/0a3f50b9-5c39-32b8-e044-0003ba298018,fejl,fejl,fejl,fejl
"Selmers Vænge 4,2970",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,fejl,fejl,fejl,NaN,fejl,https://www.boligsiden.dk/addresses/0a3f50aa-1f5e-32b8-e044-0003ba298018,fejl,fejl,fejl,fejl
"Munkebjergvej 84,2770",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,fejl,fejl,fejl,NaN,fejl,https://www.boligsiden.dk/addresses/0a3f50a7-57be-32b8-e044-0003ba298018,fejl,fejl,fejl,fejl


In [30]:
#Converts to csv-file and save in "Rawdata"
df_DF_NAME.to_csv(f'rawdata/{DF_NAME}.csv', index=True)

